In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import tensorflow_text

In [2]:
USE_module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3')
#USE_CNN_module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')

In [3]:
data = pd.read_csv("/Users/kinaida/Documents/Capstone_Data/Crea-OFT/FleurThesis/CreateData/RequiredData/aut_categorizedRaters.csv", encoding = 'latin-1')

#data = data.drop(["research_id", "response_id", "respondent_id", "object"], axis = 1)

data.head()

,research_id,response_id,respondent_id,object,original_response,cleaned_response,C0,C1,C2,C3,...,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63
0,CES201610,10063.0,648,brick,aambeeld,aambeeld,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CES201610,5801.0,363,brick,het aanleggen van een weg,aanleggen weg,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,CES201610,3973.0,237,brick,aannemen,aannemen,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CES201610,5226.0,300,brick,aannemer,aannemer,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,CES201610,8321.0,555,brick,aarde,aarde,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
orig_resp = data[['original_response']].to_numpy()

embeddings = USE_module(orig_resp)
#CNN_embeddings = USE_CNN_module(orig_resp)

In [5]:
data.insert(6, "USE_embeddings", embeddings)
#data.insert(7, "USE_CNN_embeddings", CNN_embeddings)
data.head()

,research_id,response_id,respondent_id,object,original_response,cleaned_response,USE_embeddings,C0,C1,C2,...,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63
0,CES201610,10063.0,648,brick,aambeeld,aambeeld,"(tf.Tensor(-0.0033594358, shape=(), dtype=floa...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CES201610,5801.0,363,brick,het aanleggen van een weg,aanleggen weg,"(tf.Tensor(-0.042379927, shape=(), dtype=float...",0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,CES201610,3973.0,237,brick,aannemen,aannemen,"(tf.Tensor(-0.049748804, shape=(), dtype=float...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CES201610,5226.0,300,brick,aannemer,aannemer,"(tf.Tensor(-0.035619646, shape=(), dtype=float...",0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,CES201610,8321.0,555,brick,aarde,aarde,"(tf.Tensor(-0.0018415087, shape=(), dtype=floa...",1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

numeric_cols = data.loc[:, 'C0':'C63']

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(numeric_cols.corr(), cmap='YlOrRd', fmt='.2f', vmin=0)

In [ ]:
#data.columns = ["Original_response", "Cleaned_response","Overig","Bouwen","Gooien","Vandalisme","Agressie","Meubels","Huis_Accessoires","Tekenen/ Schrijven","Kleding","Voedsel","Cosmetica_Accessoire",
#"Gereedschap","Technologie","Component","Muziekinstrument/ Geluid_maken","Voertuig","Item_met_emotionele_waarde","Kunst","Verkeer",
#"Game","Steunen","Blokkeren","Territorium_markeren","Kapotslaan/ Breken","Snijden","Opvullen","Als gewicht","Bescherming","Verhoging",
#"Sporten","Therapie","Plagen","Hitte/Vuur","Ergens_instoppen","Fantasie","Schoonmaken","Ruilen/geld","Wetenschap","In_de_keuken","Accessoire_voor_tuin",
#"Bedekken/Afsluiten","Accessoire voor dier","Feestdagen","Verjaardagen","Graven","Moord","Overleven","Balanceren","Versieren","Beeldhouwen",
#"Wapen","Speelgoed","Sociale_interactie","Hulpmiddel","Sfeer","Pletten","Baan/Werk","Kleur","Vermalen","Knutselen","Vermaak", "Natuur",
#"Testen","Water_tool", "USE_embeddings"]

#data.head()

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hypermodel import HyperModel
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.model_selection import train_test_split

In [9]:
X = embeddings.numpy()
Y = data.iloc[:,7:71].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 0)

In [10]:
import statistics
sm = int()
mn = int()

def row_sum(predictions, test):
    sm = statistics.mean((predictions * test).sum(axis = 1) / test.sum(axis = 1))
    return sm
    
def row_mean(predictions, test):
    mn = statistics.mean((predictions == test).mean(axis = 1))
    return mn

In [11]:
# Define the model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape = (512, )))
    # Tune the number of units looping over 2 till 6 Dense layers
    # Choose an optimal value for every layer between 32-800
    for i in range(hp.Int('num_layers', 2, 4)):
        model.add(layers.Dense(units = hp.Int('units_' + str(i),
                                            min_value = 32,
                                            max_value = 800,
                                            step = 32),
                               activation = 'relu'))
    # Add final output layer with sigmoid activation
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation = 'sigmoid'))
    
    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    model.compile(
        optimizer = keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss = 'binary_crossentropy',
        metrics = ['categorical_accuracy'])
    return model

# Initiate the tuner
tuner = RandomSearch(
    build_model,
    objective = 'val_categorical_accuracy',
    max_trials = 5,
    executions_per_trial = 3,
    directory = '/Users/kinaida/Documents/Capstone_Data/MLC',
    project_name = 'multi-label-classification')

tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 800, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 800, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [12]:
# Search for best model
tuner.search(X_train, Y_train, validation_data = (X_test, Y_test), epochs = 100, batch_size = 32)

Trial 5 Complete [00h 03m 36s]
val_categorical_accuracy: 0.7099999984105428

Best val_categorical_accuracy So Far: 0.7127777536710104
Total elapsed time: 00h 29m 11s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [13]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}. The optimal number
of units in the first densely-connected layer is {best_hps.get('units_1')}. The optimal number
of units in the second densely-connected layer is {best_hps.get('units_2')}. The optimal number
of units in the third densely-connected layer is {best_hps.get('units_3')} Final, the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")
print(tuner.results_summary(num_trials = 1))


The hyperparameter search is complete. The optimal number of layers is 2. The optimal number
of units in the first densely-connected layer is 704. The optimal number
of units in the second densely-connected layer is 32. The optimal number
of units in the third densely-connected layer is 800 Final, the optimal learning rate for the optimizer
is 0.001.

Results summary
Results in /Users/kinaida/Documents/Capstone_Data/MLC/multi-label-classification
Showing 1 best trials
Objective(name='val_categorical_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
units_0: 800
units_1: 704
learning_rate: 0.001
units_2: 32
units_3: 800
Score: 0.7127777536710104
None


In [14]:
# Build the model with the optimal hyperparameters and train it on the data
model   = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, validation_data = (X_test, Y_test), 
                    epochs = 100, batch_size = 64)


Epoch 1/100
85/85 [==============================] - 2s 14ms/step - loss: 0.1377 - categorical_accuracy: 0.2819 - val_loss: 0.0704 - val_categorical_accuracy: 0.4533
Epoch 2/100
85/85 [==============================] - 1s 13ms/step - loss: 0.0591 - categorical_accuracy: 0.5130 - val_loss: 0.0508 - val_categorical_accuracy: 0.5700
Epoch 3/100
85/85 [==============================] - 1s 13ms/step - loss: 0.0458 - categorical_accuracy: 0.6026 - val_loss: 0.0441 - val_categorical_accuracy: 0.6133
Epoch 4/100
85/85 [==============================] - 1s 11ms/step - loss: 0.0397 - categorical_accuracy: 0.6370 - val_loss: 0.0416 - val_categorical_accuracy: 0.6167
Epoch 5/100
85/85 [==============================] - 1s 12ms/step - loss: 0.0358 - categorical_accuracy: 0.6707 - val_loss: 0.0385 - val_categorical_accuracy: 0.6417
Epoch 6/100
85/85 [==============================] - 1s 12ms/step - loss: 0.0325 - categorical_accuracy: 0.6950 - val_loss: 0.0373 - val_categorical_accuracy: 0.6783
Epoc

85/85 [==============================] - 1s 11ms/step - loss: 0.0060 - categorical_accuracy: 0.8500 - val_loss: 0.0522 - val_categorical_accuracy: 0.6967
Epoch 51/100
85/85 [==============================] - 1s 10ms/step - loss: 0.0059 - categorical_accuracy: 0.8446 - val_loss: 0.0506 - val_categorical_accuracy: 0.6800
Epoch 52/100
85/85 [==============================] - 1s 12ms/step - loss: 0.0056 - categorical_accuracy: 0.8500 - val_loss: 0.0515 - val_categorical_accuracy: 0.6900
Epoch 53/100
85/85 [==============================] - 1s 11ms/step - loss: 0.0059 - categorical_accuracy: 0.8472 - val_loss: 0.0528 - val_categorical_accuracy: 0.7000
Epoch 54/100
85/85 [==============================] - 1s 11ms/step - loss: 0.0058 - categorical_accuracy: 0.8487 - val_loss: 0.0524 - val_categorical_accuracy: 0.6850
Epoch 55/100
85/85 [==============================] - 1s 11ms/step - loss: 0.0056 - categorical_accuracy: 0.8548 - val_loss: 0.0520 - val_categorical_accuracy: 0.6950
Epoch 56/10

85/85 [==============================] - 1s 11ms/step - loss: 0.0048 - categorical_accuracy: 0.8496 - val_loss: 0.0588 - val_categorical_accuracy: 0.7000
Epoch 100/100
85/85 [==============================] - 1s 14ms/step - loss: 0.0047 - categorical_accuracy: 0.8478 - val_loss: 0.0575 - val_categorical_accuracy: 0.6883


In [15]:
# now we can test the accuracy of the model on the test data
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss

# make model predictions
Y_pred         = model.predict(X_test)
thresholds     = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for val in thresholds:
    predictions       = Y_pred.copy()
  
    predictions[predictions >= val] = 1
    predictions[predictions < val]  = 0

    precision   = precision_score(Y_test, predictions, average = 'micro')
    recall      = recall_score(Y_test, predictions, average = 'micro')
    f1          = f1_score(Y_test, predictions, average = 'micro')
    hamming     = hamming_loss(Y_test, predictions)
    sum_metric  = row_sum(predictions, Y_test)
    mean_metric = row_mean(predictions, Y_test)
   
    print("Micro-average quality numbers")
    print("Threshold: {:.1f}".format(val))
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}, Hamming loss: {:.4f}, Sum Metric: {:.4f}, Mean Metric: {:.4f}".format(precision, 
                                                                                               recall, f1, hamming, sum_metric, mean_metric))

Micro-average quality numbers
Threshold: 0.1
Precision: 0.6877, Recall: 0.7812, F1-measure: 0.7315, Hamming loss: 0.0121, Sum Metric: 0.8051, Mean Metric: 0.9879
Micro-average quality numbers
Threshold: 0.2
Precision: 0.7190, Recall: 0.7528, F1-measure: 0.7355, Hamming loss: 0.0114, Sum Metric: 0.7797, Mean Metric: 0.9886
Micro-average quality numbers
Threshold: 0.3
Precision: 0.7392, Recall: 0.7219, F1-measure: 0.7305, Hamming loss: 0.0112, Sum Metric: 0.7542, Mean Metric: 0.9888
Micro-average quality numbers
Threshold: 0.4
Precision: 0.7633, Recall: 0.6934, F1-measure: 0.7267, Hamming loss: 0.0110, Sum Metric: 0.7289, Mean Metric: 0.9890
Micro-average quality numbers
Threshold: 0.5
Precision: 0.7803, Recall: 0.6675, F1-measure: 0.7195, Hamming loss: 0.0110, Sum Metric: 0.7083, Mean Metric: 0.9890
Micro-average quality numbers
Threshold: 0.6
Precision: 0.7960, Recall: 0.6465, F1-measure: 0.7135, Hamming loss: 0.0109, Sum Metric: 0.6919, Mean Metric: 0.9891
Micro-average quality number

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [19]:
# Dependent variables
Y = data.iloc[:, 7:71]

# Independent variables
X = embeddings.numpy()

# Split data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 0)

# corresponding responses to test set
corr_response = data.iloc[Y_test.index, 5].reset_index(drop = True)
co_response   = data.iloc[Y_test.index, 5]

# create a dataframe for the predictions by the neural network
df = pd.DataFrame(data = Y_pred[0:,0:],
                  index = [i for i in range(Y_pred.shape[0])],
                  columns = ['C' + str(i) for i in range(Y_pred.shape[1])])

pred_data = pd.concat([corr_response, df], axis = 1)
true_data = pd.concat([co_response, Y_test], axis = 1).reset_index(drop = True)

# Write to csv files
pred_data.to_csv('algorithm_predictions.csv')
true_data.to_csv('true_data.csv')

In [20]:
from sklearn.metrics import hamming_loss

# Load all raters
rater01      = pd.read_csv("data_rater1.csv", encoding = 'latin-1')
rater02      = pd.read_csv("data_rater2.csv", encoding = 'latin-1')
rater03      = pd.read_csv("data_rater3.csv", encoding = 'latin-1')

# Rater 1 vs Rater 2
rater01_02   = hamming_loss(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71])
precision1   = precision_score(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71], average = 'micro')
recall1      = recall_score(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71], average = 'micro')
f11          = f1_score(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71], average = 'micro')
sum_metric1  = row_sum(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71])
mean_metric1 = row_mean(rater01.iloc[:, 6:71], rater02.iloc[:, 6:71])

# Rater 1 vs Rater 3
rater01_03   = hamming_loss(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71])
precision2   = precision_score(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
recall2      = recall_score(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
f12          = f1_score(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
sum_metric2  = row_sum(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71])
mean_metric2 = row_mean(rater01.iloc[:, 6:71], rater03.iloc[:, 6:71])

# Rater 2 vs Rater 3
rater02_03   = hamming_loss(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71])
precision3   = precision_score(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
recall3      = recall_score(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
f13          = f1_score(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71], average = 'micro')
sum_metric3  = row_sum(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71])
mean_metric3 = row_mean(rater02.iloc[:, 6:71], rater03.iloc[:, 6:71])

print("Rater 1 vs Rater 2:")
print("Hamming Loss:", round(rater01_02, 3))
print("Precision:", round(precision1, 3))
print("Recall:", round(recall1, 3))
print("F-1:", round(f11, 3))
print("Sum Metric:", round(sum_metric1, 3))
print("Mean Metric:", round(mean_metric1, 3))

print("Rater 1 vs Rater 3:")
print("Hamming Loss:", round(rater01_03, 3))
print("Precision:", round(precision2, 3))
print("Recall:", round(recall2, 3))
print("F-1:", round(f12, 3))
print("Sum Metric:", round(sum_metric2, 3))
print("Mean Metric:", round(mean_metric2, 3))

print("Rater 2 vs Rater 3:")
print("Hamming Loss:", round(rater02_03, 3))
print("Precision:", round(precision3, 3))
print("Recall:", round(recall3, 3))
print("F-1:", round(f13, 3))
print("Sum Metric:", round(sum_metric3, 3))
print("Mean Metric:", round(mean_metric3, 3))

FileNotFoundError: [Errno 2] File b'data_rater1.csv' does not exist: b'data_rater1.csv'

In [ ]:
# Select same rows for the raters as for the test data from the algorithm
rat01_alg    = rater01.iloc[y_test.index, 6:71]
rat02_alg    = rater02.iloc[y_test.index, 6:71]
rat03_alg    = rater03.iloc[y_test.index, 6:71]

# Rater 1 vs Algorithm
rater01_test = hamming_loss(rat01_alg, y_test)

# Rater 1 vs Algorithm
rater02_test = hamming_loss(rat02_alg, y_test)

# Rater 2 vs Algorithm
rater03_test = hamming_loss(rat03_alg, y_test)

print("Rater 1 vs Algorithm:", rater01_test)
print("Rater 2 vs Algorithm:", rater02_test)
print("Rater 3 vs Algorithm:", rater03_test)